## Midi Predictor

In [1]:
import numpy as np

from htm.bindings.sdr import SDR
from htm.bindings.encoders import ScalarEncoder, ScalarEncoderParameters
from htm.algorithms import SpatialPooler as SP
from htm.algorithms import TemporalMemory as TM
from htm.bindings.algorithms import Predictor

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [2]:
scalarEncoderParams = ScalarEncoderParameters()
scalarEncoderParams.minimum = 59
scalarEncoderParams.maximum = 72
scalarEncoderParams.activeBits = 3
scalarEncoderParams.size = 24
scalarEncoderParams.clipInput  = True

enc = ScalarEncoder(scalarEncoderParams)

In [3]:
inputSDR  = SDR( dimensions = (24, ) )
activeSDR = SDR( dimensions = (576,) )
sp = SP(inputDimensions  = inputSDR.dimensions,
        columnDimensions = activeSDR.dimensions,
        localAreaDensity = 0.02,
        globalInhibition = True,
        seed             = 1,
        synPermActiveInc   = 0.01,
        synPermInactiveDec = 0.008)

In [4]:
tm = TM(
    columnDimensions = (576,),
    cellsPerColumn=8,
    initialPermanence=0.5,
    connectedPermanence=0.5,
    minThreshold=8,
    maxNewSynapseCount=20,
    permanenceIncrement=0.1,
    permanenceDecrement=0.0,
    activationThreshold=8,
)

In [5]:
seq =[]
twinkle = [60,60,67,67,69,69,67,65,65,64,64,62,62,60]
for i in range(3):
    seq.extend(twinkle)

predictor = Predictor( steps=[1, 2,3,4,5,6,7], alpha=0.1)
predictor_resolution = 1

In [6]:
for i in range(len(seq)):
    print(seq[i])
    inputSDR = enc.encode(seq[i])
    sp.compute(inputSDR, True, activeSDR)

    tm.compute( activeSDR, learn=True)
    
    predictor.learn(i, tm.getActiveCells(), int(seq[i] / predictor_resolution))

    pdf = predictor.infer( tm.getActiveCells() )
    for n in (1,2,3,4,5,6,7):
        if pdf[n]:
            print(n, np.argmax( pdf[n] ) * predictor_resolution )
        else:
            print(n, float('nan'))

60
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
60
1 60
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
67
1 0
2 0
3 nan
4 nan
5 nan
6 nan
7 nan
67
1 67
2 0
3 0
4 nan
5 nan
6 nan
7 nan
69
1 0
2 0
3 0
4 0
5 nan
6 nan
7 nan
69
1 69
2 0
3 0
4 0
5 0
6 nan
7 nan
67
1 69
2 69
3 67
4 67
5 0
6 0
7 nan
65
1 0
2 0
3 0
4 0
5 0
6 0
7 0
65
1 65
2 0
3 0
4 0
5 0
6 0
7 0
64
1 67
2 65
3 65
4 64
5 64
6 0
7 0
64
1 64
2 65
3 65
4 64
5 64
6 64
7 0
62
1 62
2 62
3 0
4 0
5 0
6 0
7 0
62
1 62
2 62
3 62
4 0
5 0
6 0
7 0
60
1 67
2 67
3 69
4 69
5 67
6 65
7 65
60
1 60
2 67
3 69
4 69
5 67
6 65
7 65
60
1 60
2 60
3 69
4 69
5 67
6 65
7 65
67
1 65
2 65
3 64
4 64
5 62
6 62
7 60
67
1 65
2 65
3 64
4 64
5 62
6 62
7 60
69
1 67
2 65
3 65
4 64
5 64
6 62
7 62
69
1 67
2 65
3 65
4 64
5 64
6 62
7 62
67
1 65
2 65
3 64
4 64
5 62
6 62
7 60
65
1 64
2 64
3 62
4 62
5 60
6 60
7 60
65
1 64
2 64
3 62
4 62
5 60
6 60
7 60
64
1 62
2 62
3 60
4 60
5 60
6 67
7 67
64
1 62
2 62
3 60
4 60
5 60
6 67
7 67
62
1 60
2 60
3 60
4 67
5 67
6 69
7 69
62
1 60
2 60
3 60
4 67


In [6]:
predictions = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: []}
for i in range(len(seq)):
    inputSDR = enc.encode(seq[i])
    sp.compute(inputSDR, True, activeSDR)

    tm.compute( activeSDR, learn=True)
    
    predictor.learn(i, tm.getActiveCells(), int(seq[i] / predictor_resolution))

    pdf = predictor.infer( tm.getActiveCells() )
    for n in (1,2,3,4,5,6,7):
        if pdf[n]:
            predictions[n].append( np.argmax( pdf[n] ) * predictor_resolution )
        else:
            predictions[n].append(float('nan'))
print(predictions)

{1: [nan, 60, 0, 67, 0, 69, 69, 0, 65, 67, 64, 62, 62, 67, 60, 60, 65, 65, 67, 67, 65, 64, 64, 62, 62, 60, 60, 60, 60, 60, 67, 69, 69, 67, 65, 65, 64, 64, 62, 62, 60, 60], 2: [nan, nan, 0, 0, 0, 0, 69, 0, 0, 65, 65, 62, 62, 67, 67, 60, 65, 65, 65, 65, 65, 64, 64, 62, 62, 60, 60, 60, 67, 67, 69, 69, 67, 65, 65, 64, 64, 62, 62, 60, 60, 60], 3: [nan, nan, nan, 0, 0, 0, 67, 0, 0, 65, 65, 0, 62, 69, 69, 69, 64, 64, 65, 65, 64, 62, 62, 60, 60, 60, 60, 67, 67, 67, 69, 67, 65, 65, 64, 64, 62, 62, 60, 60, 60, 67], 4: [nan, nan, nan, nan, 0, 0, 67, 0, 0, 64, 64, 0, 0, 69, 69, 69, 64, 64, 64, 64, 64, 62, 62, 60, 60, 67, 67, 67, 69, 69, 67, 65, 65, 64, 64, 62, 62, 60, 60, 60, 67, 67], 5: [nan, nan, nan, nan, nan, 0, 0, 0, 0, 64, 64, 0, 0, 67, 67, 67, 62, 62, 64, 64, 62, 60, 60, 60, 60, 67, 67, 69, 69, 69, 65, 65, 64, 64, 62, 62, 60, 60, 60, 67, 67, 69], 6: [nan, nan, nan, nan, nan, nan, 0, 0, 0, 0, 64, 0, 0, 65, 65, 65, 62, 62, 62, 62, 62, 60, 60, 67, 67, 69, 69, 69, 65, 65, 65, 64, 64, 62, 62, 60

In [11]:
[60,60,67,67,69,69,67,65,65,64,64,62,62,60]

[60, 60, 67, 67, 69, 69, 67, 65, 65, 64, 64, 62, 62, 60]

In [ ]:
[60,67,67,67,69,69,67,65,65,64,64,62,62,60]